In [19]:
!pip install PyTDC smiles-encoder scipy

17988.81s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [20]:
!pip install scikit-learn

17998.02s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [3]:
from tdc.single_pred import Tox
from sklearn import svm

data = Tox(name = 'hERG')
df = data.get_data()
df


#https://pypi.org/project/smiles-encoder/ smile encoder
#workflow
# encode SMILES strings --> binary classification
#timeline
#gather packages needed
#have funtional model
# refine model parameters
# create

Found local copy...
Loading...
Done!


,Drug_ID,Drug,Y
0,DEMETHYLASTEMIZOLE,Oc1ccc(CCN2CCC(Nc3nc4ccccc4n3Cc3ccc(F)cc3)CC2)cc1,1.0
1,GBR-12909,Fc1ccc(C(OCC[NH+]2CC[NH+](CCCc3ccccc3)CC2)c2cc...,1.0
2,LY-97241,CCCCCCCN(CC)CCCCc1ccc([N+](=O)[O-])cc1,1.0
3,CLOFILIUM PHOSPHATE,CCCCCCC[N+](CC)(CC)CCCCc1ccc(Cl)cc1.CCCCCCC[N+...,1.0
4,FLUSPIRILENE,O=C1NCN(c2ccccc2)C12CC[NH+](CCCC(c1ccc(F)cc1)c...,1.0
...,...,...,...
650,THEOPHYLLINE,Cn1c(=O)c2nc[nH]c2n(C)c1=O,0.0
651,THIOGUANINE,Nc1nc(=S)c2[nH]cnc2[nH]1,0.0
652,TILUDRONIC_ACID,O=P([O-])([O-])C(Sc1ccc(Cl)cc1)P(=O)([O-])[O-],0.0
653,TIZANIDINE,Clc1ccc2nsnc2c1NC1=[NH+]CCN1,0.0


In [4]:
split = data.get_split()
smiles_list_train = split['train']['Drug'].tolist()
Y_train = split['train']['Y']
Y_train

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
453    0.0
454    0.0
455    0.0
456    0.0
457    0.0
Name: Y, Length: 458, dtype: float64

In [35]:
from smiles_encoder import SmilesEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np


#encode into vectors using one hot encoding 
encoder = SmilesEncoder(smiles_list_train)
X_train = encoder.encode_many(smiles_list_train)
encoder_valid = SmilesEncoder(split['valid']['Drug'].tolist())
X_valid = encoder_valid.encode_many(split['valid']['Drug'].tolist())
Y_train = split['train']['Y']


# Pad sequences to a fixed length
# Step 1: Determine maximum sizes
max_outer = max(max(len(sample) for sample in X_train), max(len(sample) for sample in X_valid))  # Outer dimension
max_inner = max(
    max(len(inner) for sample in X_train for inner in sample),
    max(len(inner) for sample in X_valid for inner in sample),
)  # Inner dimension

# Step 2: Function to pad both outer and inner dimensions
def pad_samples(data, max_outer, max_inner):
    padded = []
    for sample in data:
        # Pad inner lists to max_inner
        padded_sample = [inner + [0] * (max_inner - len(inner)) for inner in sample]
        # Pad the outer list to max_outer with zero-filled inner lists
        padded_sample += [[0] * max_inner] * (max_outer - len(sample))
        padded.append(padded_sample)
    return np.array(padded)
# Apply padding to both datasets
X_train_padded = pad_samples(X_train, max_outer, max_inner)
X_val_padded = pad_samples(X_valid, max_outer, max_inner)


#flatten 
X_train_final = [np.array(sample).flatten() for sample in X_train_padded]

#train SVM
svm = SVC()
param_grid = {'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(estimator=svm, param_grid=param_grid, scoring='accuracy', cv=5)
grid.fit(X_train_final, Y_train)

# Evaluate
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)



Best parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Best cross-validation score: 0.7928810320114668


In [36]:

X_valid_final = [np.array(sample).flatten() for sample in X_val_padded]
Y_valid = split['valid']['Y']

# Evaluate on validation data
best_model = grid.best_estimator_
Y_pred_valid = best_model.predict(X_valid_final)

# Validation metrics
print("Validation accuracy:", accuracy_score(Y_valid, Y_pred_valid))
print("Classification report:\n", classification_report(Y_valid, Y_pred_valid))

Validation accuracy: 0.48484848484848486
Classification report:
               precision    recall  f1-score   support

         0.0       0.38      0.74      0.50        23
         1.0       0.71      0.35      0.47        43

    accuracy                           0.48        66
   macro avg       0.55      0.54      0.48        66
weighted avg       0.60      0.48      0.48        66

